In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch.nn import functional as F
import torch.optim as  optim
import torchvision
import torchvision.transforms as transforms

In [25]:
# Process for this project. Load the data, training and testing the data, train and evaluate the model

In [7]:
# Data Transformation and Normalization

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [9]:
# Loading CIFAR-10 Dataset
# you can use any CIFAR your choice
train_data = torchvision.datasets.CIFAR10(root='.data/', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='.data/', train=False, transform=transform, download=True)

#DataLoader for Batching
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100%|████████████████████████████████████████████████████████████████| 170498071/170498071 [03:45<00:00, 756947.37it/s]


Extracting .data/cifar-10-python.tar.gz to .data/
Files already downloaded and verified


In [10]:
#Sample Data Inspection
image, label = train_data[0]

In [12]:
image.size()

torch.Size([3, 32, 32])

In [13]:
# Lets create the classes for the image. 
# These are labels for our 10 classes. Our network will generate the number btw 0-9 rep individual classes
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [20]:
# Neural Network Definition
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5)  # (24, 10, 10) -> (24, 5, 5) -> Flatten(24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)  # Corrected to include output features
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Corrected flattening method
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [21]:
# Let's Setup Our Training
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [22]:
# Training Loop
for epoch in range(30):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'loss: {running_loss / len(train_loader):.4f}')


Training epoch 0...
loss: 2.1294
Training epoch 1...
loss: 1.7483
Training epoch 2...
loss: 1.5124
Training epoch 3...
loss: 1.4028
Training epoch 4...
loss: 1.3130
Training epoch 5...
loss: 1.2313
Training epoch 6...
loss: 1.1620
Training epoch 7...
loss: 1.1026
Training epoch 8...
loss: 1.0478
Training epoch 9...
loss: 1.0053
Training epoch 10...
loss: 0.9620
Training epoch 11...
loss: 0.9239
Training epoch 12...
loss: 0.8917
Training epoch 13...
loss: 0.8604
Training epoch 14...
loss: 0.8277
Training epoch 15...
loss: 0.8023
Training epoch 16...
loss: 0.7730
Training epoch 17...
loss: 0.7460
Training epoch 18...
loss: 0.7211
Training epoch 19...
loss: 0.6961
Training epoch 20...
loss: 0.6734
Training epoch 21...
loss: 0.6471
Training epoch 22...
loss: 0.6245
Training epoch 23...
loss: 0.6027
Training epoch 24...
loss: 0.5813
Training epoch 25...
loss: 0.5600
Training epoch 26...
loss: 0.5394
Training epoch 27...
loss: 0.5188
Training epoch 28...
loss: 0.4981
Training epoch 29...
los

In [23]:
# Saving and Loading the Model
torch.save(net.state_dict(), 'trained_net.pth')

In [24]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [25]:
# Model Evaluation
correct = 0
total = 0 

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct +=(predicted == labels).sum().item() #how many has the correct classification

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')
        

Accuracy: 67.57%


In [35]:
# Image Preprocessing for New Images
new_transform = transforms.Compose([

    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

# Image Prediction
image_path = ['image1.jpg', 'image2.jpg'] 
images = [load_image(img) for img in image_path]

net.eval()
with torch.no_grad():
    for image in images:
        outputs = net(image)
        _, predicted = torch.max(outputs, 1)
        print(f'Predicted: {class_names[predicted.item()]}')

Predicted: ship
Predicted: plane
